In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import json
import folium

from sklearn import preprocessing 

In [ ]:
# apply, concat, merge, melt, pivot, pivot_table, groupby

In [ ]:
titanic = sns.load_dataset('titanic')
df = titanic.loc[:, ['age', 'fare']]

In [ ]:
df

,age,fare
0,22.0,7.2500
1,38.0,71.2833
2,26.0,7.9250
3,35.0,53.1000
4,35.0,8.0500
...,...,...
886,27.0,13.0000
887,19.0,30.0000
888,NaN,23.4500
889,26.0,30.0000


In [ ]:
def add_10(n):
  return n + 10

In [ ]:
df_map = df.applymap(add_10)
print(df_map.head(3))

    age     fare
0  32.0  17.2500
1  48.0  81.2833
2  36.0  17.9250


In [ ]:
def missing_value(series):
    return series.isnull()  #boolean 시리즈로 변환

In [ ]:
result = df.apply(missing_value, axis = 0)  # function is in apply
print(result.head(3))

     age   fare
0  False  False
1  False  False
2  False  False


In [ ]:
def min_max(x):
   return x.max() - x.min()

In [ ]:
result2 = df.apply(min_max)   # default => axis = 0
print(result2)

age      79.5800
fare    512.3292
dtype: float64


In [ ]:
df['ten'] = 10 

In [ ]:
def add_two_obj(a, b):
   return a + b

In [ ]:
# 데이터프레임의 2개열을 선택해서 적용
df['add'] = df.apply(lambda x: add_two_obj(x['age'], x['ten']), axis = 1)
#DataFrame.apply(func, axis=0, raw=False, result_type=None, args=(), **kwargs)
df.head(3)

,age,fare,ten,add
0,22.0,7.2500,10,32.0
1,38.0,71.2833,10,48.0
2,26.0,7.9250,10,36.0


In [ ]:
from pandas.io.parsers.readers import read_csv
filename = './서울시.csv'

cctv= pd.read_csv(filename, skiprows=1, encoding='EUC-KR') # encoding ='EUC-KR'
cctv

FileNotFoundError: ignored

In [ ]:
cctv['총계'] = cctv['총계'].apply(lambda x: int(x.replace(',', ' ')))
cctv

In [ ]:
# 결측치를 0으로 대체, 정수로 변환
cctv.isna().sum()

In [ ]:
cctv['2012년'].fillna('0', inplace=True)
cctv['2013년'].fillna('0', inplace=True)

In [ ]:
for col in cctv.columns[2:]:
   cctv[col] = cctv[col].apply(lambda x: int(x.replace('.', '')))

In [ ]:
cctv.info()

In [ ]:
titanic = sns.load_dataset('titanic')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [ ]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [ ]:
# 1.age를 평균으로 대체  
# 2.embarked 최빈값으로 대체 
# 3.embark_town을 결측값이 있는 행 삭제
# 4.deck 삭제
# 5.titanic['adult/child'] 컬럼 만들고 20살 이상이면 adult 이하면 child
# apply, lambda 활용
# 6. bins = [1, 20, 30,50,70,100]
#  label = ['미성년자', '청년', '중년', '장년', '노년'] 
#  age를 기준으로 'age_cut' 컬럼을 생성후 두 컬럼을 출력하기(cut)
#  age, age_cut
#  22   청년
#  38   중년

In [ ]:
df = titanic

In [ ]:
# 1. 
mean_age = df['age'].mean(axis=0)        # age 열의 평균을 계산 (NAN 값을 제외)
df['age'].fillna(mean_age, inplace=True)
df['age']

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: age, Length: 891, dtype: float64

In [ ]:
# 2. 
most_freq = df['embarked'].value_counts(dropna = True).idxmax()
print(most_freq)
df['embarked'].fillna(most_freq, inplace=True)
df['embarked']

S


0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: embarked, Length: 891, dtype: object

In [ ]:
# 3.
# df['embark_town'].dropna(how='any',inplace=True)
# df['embark_town']

In [ ]:
# solution
df.dropna(subset=['embark_town'], inplace=True)
df['embark_town'].isnull().sum()

0

In [ ]:
# 4. 
df.drop(['deck'], axis=1, inplace = True)
df.shape  # 15 -> 14

(889, 14)

In [ ]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'embark_town', 'alive',
       'alone', 'age_cut'],
      dtype='object')

In [ ]:
# titanic['adult/child'], 20살 이상이면 adult 이하면 child
for i in range(len(df)):
    if df['age'][i] >= 20:
      df['adult/child'][i] = 'adult'
    else: 
      df['adult/child'][i] = 'child'

df

KeyError: ignored

In [ ]:
# solution
df['adult/child'] = df.age.apply(lambda x: 'adult'
                                if x >20 else 'child'
                                )
df

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone,age_cut,adult/child
0,0,3,male,22.000000,1,0,7.2500,S,Third,man,True,Southampton,no,False,청년,adult
1,1,1,female,38.000000,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False,중년,adult
2,1,3,female,26.000000,0,0,7.9250,S,Third,woman,False,Southampton,yes,True,청년,adult
3,1,1,female,35.000000,1,0,53.1000,S,First,woman,False,Southampton,yes,False,중년,adult
4,0,3,male,35.000000,0,0,8.0500,S,Third,man,True,Southampton,no,True,중년,adult
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.000000,0,0,13.0000,S,Second,man,True,Southampton,no,True,청년,adult
887,1,1,female,19.000000,0,0,30.0000,S,First,woman,False,Southampton,yes,True,미성년자,child
888,0,3,female,29.699118,1,2,23.4500,S,Third,woman,False,Southampton,no,False,청년,adult
889,1,1,male,26.000000,0,0,30.0000,C,First,man,True,Cherbourg,yes,True,청년,adult


In [ ]:
# 6.  bins = [1, 20, 30,50,70,100]
#  label = ['미성년자', '청년', '중년', '장년', '노년'] 
#  age를 기준으로 'age_cut' 컬럼을 생성후 두 컬럼을 출력하기(cut)
#  age, age_cut
#  22 청년
#  38 중년

In [ ]:
bins = [1, 20, 30,50,70,100]
label = ['미성년자', '청년', '중년', '장년', '노년'] 

In [ ]:
df['age_cut'] = pd.cut(x=df['age'], 
                      bins = bins,         # 경계값 리스트
                      labels = label,      # 구간별 이름
                      include_lowest=True) # 첫 경계값을 포함

In [ ]:
df[['age', 'age_cut']]

,age,age_cut
0,22.000000,청년
1,38.000000,중년
2,26.000000,청년
3,35.000000,중년
4,35.000000,중년
...,...,...
886,27.000000,청년
887,19.000000,미성년자
888,29.699118,청년
889,26.000000,청년


# 데이터프레임 연결

In [ ]:
# pd.concat(데이터프레임 리스트,  axis = 축)
df1 = pd.DataFrame([['a',1], ['b', 2]], columns = ['letter', 'number'])
df2 = pd.DataFrame([['c',3], ['d', 4]], columns = ['letter', 'number'])
df3 = pd.DataFrame([['e',5, '!'], ['f', 6, '@']], columns = ['letter', 'number', 'etc'])
df1

,letter,number
0,a,1
1,b,2


In [ ]:
# 컬럼명을 기준으로 연결
df_rowconcat = pd.concat([df1, df2, df3])  # axis=0 default
df_rowconcat

,letter,number,etc
0,a,1,NaN
1,b,2,NaN
0,c,3,NaN
1,d,4,NaN
0,e,5,!
1,f,6,@


In [ ]:
# 공통된 컬럼만 남기기
df_rowconcat2 = pd.concat([df1, df2, df3], join='inner')
df_rowconcat2 

,letter,number
0,a,1
1,b,2
0,c,3
1,d,4
0,e,5
1,f,6


In [ ]:
# 인덱스 재지정
df_rowconcat3 = pd.concat([df1, df2, df3], join='inner', ignore_index=True)
df_rowconcat3

,letter,number
0,a,1
1,b,2
2,c,3
3,d,4
4,e,5
5,f,6


# 열로 연결하기

In [ ]:
df4 = pd.DataFrame({'age':[20,21,22]}, index=['amy', 'james', 'david'])
df5 = pd.DataFrame({'phone':['010-1111-1111',
                             '010-2222-2222',
                             '010-3333-3333',
                             ]},
                   index = ['amy', 'james', 'david']
                   )
df6 = pd.DataFrame({'job':['student', 'programer', 'ceo', 'designer']},
                   index = ['amy', 'james', 'david', 'kim']
                   )
df4

,age
amy,20
james,21
david,22


In [ ]:
df5

,phone
amy,010-1111-1111
james,010-2222-2222
david,010-3333-3333


In [ ]:
df6

,job
amy,student
james,programer
david,ceo
kim,designer


In [ ]:
df_column_concat = pd.concat([df4, df5, df6])
df_column_concat

,age,phone,job
amy,20.0,NaN,NaN
james,21.0,NaN,NaN
david,22.0,NaN,NaN
amy,NaN,010-1111-1111,NaN
james,NaN,010-2222-2222,NaN
david,NaN,010-3333-3333,NaN
amy,NaN,NaN,student
james,NaN,NaN,programer
david,NaN,NaN,ceo
kim,NaN,NaN,designer


In [ ]:
df_column_concat = pd.concat([df4, df5, df6], axis=1)
df_column_concat

,age,phone,job
amy,20.0,010-1111-1111,student
james,21.0,010-2222-2222,programer
david,22.0,010-3333-3333,ceo
kim,NaN,NaN,designer


In [ ]:
df_column_concat = pd.concat([df4, df5, df6], axis=1, join='inner')
df_column_concat

,age,phone,job
amy,20,010-1111-1111,student
james,21,010-2222-2222,programer
david,22,010-3333-3333,ceo


https://stackoverflow.com/questions/38256104/differences-between-merge-and-concat-in-pandas


#pd.merge can take DataFrames as its argument, and is used to combine two DataFrames with same columns or index, which can't be done with pd.concat since it will show the repeated column in the DataFrame.


# merge has left, right, inner, outer join
# concat has axis = 0 or 1, inner, outer join

# merge (공통 데이터만으로 연결)

In [ ]:
# pd.merge(left, right, on=기준 컬럼, how=연결방법)
df = pd.read_csv('./scores.csv')
df

,name,kor,eng,math
0,Aiden,100.0,90.0,95.0
1,Charles,90.0,80.0,75.0
2,Danial,95.0,100.0,100.0
3,Evan,100.0,100.0,100.0
4,Henry,NaN,35.0,60.0
5,Ian,90.0,100.0,90.0
6,James,70.0,75.0,65.0
7,Julian,80.0,90.0,55.0
8,Justin,50.0,60.0,100.0
9,Kevin,100.0,100.0,90.0


In [ ]:
df.head(4)

,name,kor,eng,math
0,Aiden,100.0,90.0,95.0
1,Charles,90.0,80.0,75.0
2,Danial,95.0,100.0,100.0
3,Evan,100.0,100.0,100.0


In [ ]:
df7 = df.loc[[1,2,3]][['name', 'eng']]
df7

,name,eng
1,Charles,80.0
2,Danial,100.0
3,Evan,100.0


In [ ]:
df8 = df.loc[[1,2,4]][['name', 'math']]
df8

,name,math
1,Charles,75.0
2,Danial,100.0
4,Henry,60.0


In [ ]:
pd.merge(df7, df8, on='name')   # default: how = 'inner'

,name,eng,math
0,Charles,80.0,75.0
1,Danial,100.0,100.0


In [ ]:
pd.merge(df7, df8, on='name', how='outer')

,name,eng,math
0,Charles,80.0,75.0
1,Danial,100.0,100.0
2,Evan,100.0,NaN
3,Henry,NaN,60.0


In [ ]:
pd.merge(df7, df8, on='name', how='right') # keep right dataframe(df8)

,name,eng,math
0,Charles,80.0,75.0
1,Danial,100.0,100.0
2,Henry,NaN,60.0


# 행과 열의 형태 변경하기(melt: similar to reshape)

In [ ]:
# melt 
df.melt()

,variable,value
0,name,Aiden
1,name,Charles
2,name,Danial
3,name,Evan
4,name,Henry
...,...,...
115,math,95.0
116,math,100.0
117,math,100.0
118,math,70.0


In [ ]:
# 나열할 column를 지정하여 melt => id_vars = [열이름 리스트]
df.melt(id_vars='name')  # repeat name for each kor, eng, math. so len(df) is three times longer 

,name,variable,value
0,Aiden,kor,100.0
1,Charles,kor,90.0
2,Danial,kor,95.0
3,Evan,kor,100.0
4,Henry,kor,NaN
...,...,...,...
85,Vanessa,math,95.0
86,Viviana,math,100.0
87,Vikkie,math,100.0
88,Winnie,math,70.0


In [ ]:
df.melt(id_vars=['name', 'kor']) # only two times longer as there are two id_vars

,name,kor,variable,value
0,Aiden,100.0,eng,90.0
1,Charles,90.0,eng,80.0
2,Danial,95.0,eng,100.0
3,Evan,100.0,eng,100.0
4,Henry,NaN,eng,35.0
5,Ian,90.0,eng,100.0
6,James,70.0,eng,75.0
7,Julian,80.0,eng,90.0
8,Justin,50.0,eng,60.0
9,Kevin,100.0,eng,100.0


In [ ]:
# 행으로 위치를 변경할 열 지정
# value_vars = [열이름리스트]
df.melt(id_vars='name', value_vars='kor')

,name,variable,value
0,Aiden,kor,100.0
1,Charles,kor,90.0
2,Danial,kor,95.0
3,Evan,kor,100.0
4,Henry,kor,NaN
5,Ian,kor,90.0
6,James,kor,70.0
7,Julian,kor,80.0
8,Justin,kor,50.0
9,Kevin,kor,100.0


In [ ]:
df.melt(id_vars='name', value_vars=['kor', 'eng'])

,name,variable,value
0,Aiden,kor,100.0
1,Charles,kor,90.0
2,Danial,kor,95.0
3,Evan,kor,100.0
4,Henry,kor,NaN
5,Ian,kor,90.0
6,James,kor,70.0
7,Julian,kor,80.0
8,Justin,kor,50.0
9,Kevin,kor,100.0


In [ ]:
df.melt(id_vars='name', value_vars=['kor', 'eng'], 
        var_name='subject',
        value_name='score'
        )

,name,subject,score
0,Aiden,kor,100.0
1,Charles,kor,90.0
2,Danial,kor,95.0
3,Evan,kor,100.0
4,Henry,kor,NaN
5,Ian,kor,90.0
6,James,kor,70.0
7,Julian,kor,80.0
8,Justin,kor,50.0
9,Kevin,kor,100.0


# 열을 행으로 보내기(pivot: similar to reshape)

In [ ]:
df = df.melt(id_vars='name', var_name='subject', value_name='score')
df

,name,subject,score
0,Aiden,kor,100.0
1,Charles,kor,90.0
2,Danial,kor,95.0
3,Evan,kor,100.0
4,Henry,kor,NaN
...,...,...,...
85,Vanessa,math,95.0
86,Viviana,math,100.0
87,Vikkie,math,100.0
88,Winnie,math,70.0


In [ ]:
def get_grade(x):
   if x >= 90:
     grade = 'A'
   elif x>= 80:
     grade = 'B'
   elif x>= 70:
     grade = 'C'
   elif x>= 60:
     grade = 'D'
   else:
     grade = 'F'
   return grade

In [ ]:
df['grade'] = df['score'].apply(get_grade)
df = df.sort_values('name')
df

,name,subject,score,grade
0,Aiden,kor,100.0,A
60,Aiden,math,95.0,A
30,Aiden,eng,90.0,A
73,Amy,math,90.0,A
43,Amy,eng,75.0,C
...,...,...,...,...
28,Winnie,kor,70.0,C
88,Winnie,math,70.0,C
29,Zuly,kor,80.0,B
59,Zuly,eng,90.0,A


In [ ]:
# 데이터프레임.pivot(index = 인덱스로 사용할 컬럼,)
# columns = 컬럼으로 사용할 컬럼, values = 값으로 사용할 컬럼
# name, subject, score
df.pivot(index='name', columns='subject', values='score')

subject,eng,kor,math
name,,,
Aiden,90.0,100.0,95.0
Amy,75.0,90.0,90.0
Charles,80.0,90.0,75.0
Chloe,100.0,95.0,95.0
Danial,100.0,95.0,100.0
Danna,100.0,100.0,100.0
Ellen,60.0,NaN,NaN
Emma,65.0,70.0,70.0
Evan,100.0,100.0,100.0


In [ ]:
df.pivot(index='name', columns='subject', values='grade')

subject,eng,kor,math
name,,,
Aiden,A,A,A
Amy,C,A,A
Charles,B,A,C
Chloe,A,A,A
Danial,A,A,A
Danna,A,A,A
Ellen,D,F,F
Emma,D,C,C
Evan,A,A,A


In [ ]:
df.pivot(index='name', columns='subject', values=['score','grade'])

score               grade         
subject     eng    kor   math   eng kor math
name                                        
Aiden      90.0  100.0   95.0     A   A    A
Amy        75.0   90.0   90.0     C   A    A
Charles    80.0   90.0   75.0     B   A    C
Chloe     100.0   95.0   95.0     A   A    A
Danial    100.0   95.0  100.0     A   A    A
Danna     100.0  100.0  100.0     A   A    A
Ellen      60.0    NaN    NaN     D   F    F
Emma       65.0   70.0   70.0     D   C    C
Evan      100.0  100.0  100.0     A   A    A
Henry      35.0    NaN   60.0     F   F    D
Ian       100.0   90.0   90.0     A   A    A
James      75.0   70.0   65.0     C   C    D
Jennifer   55.0   80.0   80.0     F   B    B
Julian     90.0   80.0   55.0     A   B    F
Justin     60.0   50.0  100.0     D   F    A
Kate        NaN   50.0   50.0     F   F    F
Kevin     100.0  100.0   90.0     A   A    A
Leo        95.0   90.0   70.0     A   A    C
Linda      90.0  100.0  100.0     A   A    A
Oliver     75.0   70.0   65.0     C   C    D
Olivia     70.0   90.0   90.0     C   A    A
Peter      95.0  100.0  100.0     A   A    A
Rose       65.0   70.0   70.0     D   C    C
Sofia     100.0  100.0  100.0     A   A    A
Tiffany     NaN   90.0   90.0     F   A    A
Vanessa    70.0   95.0   95.0     C   A    A
Vikkie     50.0    NaN  100.0     F   F    A
Viviana    80.0  100.0  100.0     B   A    A
Winnie    100.0   70.0   70.0     A   C    C
Zuly       90.0   80.0   95.0     A   B    A

In [ ]:
################################################################################
################################################################################
################################################################################

In [ ]:
# 상품 판매 테이블
df = pd.DataFrame({"item": ["shirts", "shirts", "shirts", "shirts", "shirts",
                          "pants", "pants", "pants", "pants"],
                    "color": ["white", "white", "white", "black", "black",
                          "white", "white", "black", "black"],
                   "size": ["small", "large", "large", "small",
                          "small", "large", "small", "small",
                         "large"],
                   "sale": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "inventory": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,item,color,size,sale,inventory
0,shirts,white,small,1,2
1,shirts,white,large,2,4
2,shirts,white,large,2,5
3,shirts,black,small,3,5
4,shirts,black,small,3,6
5,pants,white,large,4,6
6,pants,white,small,5,8
7,pants,black,small,6,9
8,pants,black,large,7,9


In [ ]:
# 집계분석 item, size별 재고 합계를 pivot_table 로 분석
# inventory, pants 와 shirt large small

In [ ]:
df.pivot_table(index='item', columns='size', values='inventory',
               aggfunc='sum'
               )

size,large,small
item,,
pants,15,17
shirts,9,13


In [ ]:
df.pivot_table(index=['item', 'color'], columns='size', values='inventory',
               aggfunc='sum'
               )  # two indexes

size          large  small
item   color              
pants  black    9.0    9.0
       white    6.0    8.0
shirts black    NaN   11.0
       white    9.0    2.0

In [ ]:
df.pivot_table(index=['item', 'color'], columns='size', values='inventory',
               aggfunc='sum', fill_value=0
               )    # fill black large -> 0

size          large  small
item   color              
pants  black      9      9
       white      6      8
shirts black      0     11
       white      9      2

In [ ]:
df.pivot_table(index=['item', 'color'], columns='size', values=['inventory', 'sale'],
               aggfunc='sum', fill_value=0
               ) 

inventory        sale      
size             large small large small
item   color                            
pants  black         9     9     7     6
       white         6     8     4     5
shirts black         0    11     0     6
       white         9     2     4     1

In [ ]:
df_titanic = sns.load_dataset('titanic')
df_titanic = df_titanic[['survived', 'pclass', 'sex', 'age', 'embarked']]
df_titanic.dropna(inplace=True)
df_titanic

,survived,pclass,sex,age,embarked
0,0,3,male,22.0,S
1,1,1,female,38.0,C
2,1,3,female,26.0,S
3,1,1,female,35.0,S
4,0,3,male,35.0,S
...,...,...,...,...,...
885,0,3,female,39.0,Q
886,0,2,male,27.0,S
887,1,1,female,19.0,S
889,1,1,male,26.0,C


In [ ]:
# 성별, 객실등급별 승선자수 사용(count adds all)
df_titanic.pivot_table(index=['sex'], columns='pclass', values=['survived'],
               aggfunc='count', margins=True
               ) 
# ex) survived = [1,0,1,1,0] -> 5

survived               
pclass        1    2    3  All
sex                           
female       83   74  102  259
male        101   99  253  453
All         184  173  355  712

In [ ]:
# 성별, 객실등급별 생존자수 사용(sum adds 1 of survived)
df_titanic.pivot_table(index=['sex'], columns='pclass', values=['survived'],
               aggfunc='sum', margins=True
               ) 
# ex) survived = [1,0,1,1,0] -> 3

survived             
pclass        1   2   3  All
sex                         
female       80  68  47  195
male         40  15  38   93
All         120  83  85  288

In [ ]:
# 성별, 객실등급별 생존율 (mean)
df_titanic.pivot_table(index=['sex'], columns='pclass', values=['survived'],
               aggfunc='mean', margins=True
               )   # mean is default of aggfuc. (it can be removed)
# ex) survived = [1,0,1,1,0] -> 3/5

survived                              
pclass         1         2         3       All
sex                                           
female  0.963855  0.918919  0.460784  0.752896
male    0.396040  0.151515  0.150198  0.205298
All     0.652174  0.479769  0.239437  0.404494

In [ ]:
tips = sns.load_dataset('tips')
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [ ]:
tips['tip_pct'] = ((tips.tip/tips.total_bill)*100).round(2)

In [ ]:
# pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', 
# fill_value=None, margins=False, dropna=True, margins_name='All', observed=False, sort=True)

In [ ]:
tips.pivot_table(columns=['smoker'], values=['tip_pct'], index= 'sex', aggfunc='count', margins=True)

tip_pct          
smoker     Yes   No  All
sex                     
Male        60   97  157
Female      33   54   87
All         93  151  244

In [ ]:
tips.pivot_table('tip_pct', ['sex', 'smoker'])

tip_pct
sex    smoker           
Male   Yes     15.276667
       No      16.066598
Female Yes     18.214545
       No      15.691111

In [ ]:
tips.pivot_table('tip_pct', 'sex', 'smoker')

smoker,Yes,No
sex,,
Male,15.276667,16.066598
Female,18.214545,15.691111


In [ ]:
tips.pivot_table('tip_pct', 'smoker', aggfunc=['mean', 'min', 'max'])

,mean,min,max
,tip_pct,tip_pct,tip_pct
smoker,,,
Yes,16.319140,3.56,71.03
No,15.932318,5.68,29.20


# groupby

In [ ]:
# DataFrame.groupby(by=None, axis=0, level=None, as_index=True, sort=True, 
# group_keys=_NoDefault.no_default, squeeze=_NoDefault.no_default, observed=False, dropna=True)[source]

In [ ]:
# df.groupby(그룹기준컬럼), 통계적용컬럼, 통계함수
# .count(): 결측치을 제외한 데이터수
# .size(): 결측치을 포함한 데이터수
# .mean(), .sum(), .std(), .min(), .max()

In [ ]:
# 객시등급(pclass) 별 승선자수 결과 집계
df_titanic1 = df_titanic.groupby('pclass').survived.count().to_frame()
df_titanic1 

,survived
pclass,
1,184
2,173
3,355


In [ ]:
# 객실등급(pclass) 별 생존자수 결과 집계
df_titanic2 = df_titanic.groupby('pclass').survived.sum().to_frame()
df_titanic2 

,survived
pclass,
1,120
2,83
3,85


In [ ]:
# 객실등급(pclass) 별 생존률 결과 집계
df_titanic3 = df_titanic.groupby('pclass').survived.mean().to_frame()
df_titanic3 

,survived
pclass,
1,0.652174
2,0.479769
3,0.239437


In [ ]:
df_titanic4 = pd.concat([df_titanic1, df_titanic2, df_titanic3], axis=1)
df_titanic4.columns = ['승선자수', '생존자수', '생존비율']
df_titanic4 

,승선자수,생존자수,생존비율
pclass,,,
1,184,120,0.652174
2,173,83,0.479769
3,355,85,0.239437


In [ ]:
# 성별 생존통계, index = ['sex']